In [ ]:
import axs
import numpy as np
from astropy.io import fits

In [ ]:
def spark_start(project_path, metastore=None):
    from pyspark.sql import SparkSession

    warehouse_location = os.path.join(project_path, 'spark-warehouse')

    local_dir = os.path.join(project_path, 'spark-tmp')

    spark = ( 
            SparkSession.builder
            .appName("LSD2")
            .config("spark.sql.warehouse.dir", warehouse_location)
            .config('spark.master', "local[4]")
            .config('spark.driver.memory', '6G') # 128
            .config('spark.local.dir', local_dir)
            .config('spark.memory.offHeap.enabled', 'true')
            .config('spark.memory.offHeap.size', '4G') # 256
            .config("spark.sql.execution.arrow.enabled", "true")
            .config("spark.driver.maxResultSize", "6G")
            .config("spark.driver.extraJavaOptions", f"-Dderby.system.home={metastore}")
            .enableHiveSupport()
            .getOrCreate()
                    )   

    return spark

spark_session = spark_start("/epyc/users/ctslater")

catalog = axs.AxsCatalog(spark_session)

In [ ]:
sesar_df = (spark_session.read.format("csv").option("delimiter", ",")
            .load("sesar.csv", inferSchema=True, header=True))

In [ ]:
%%time
renamed_df = (sesar_df.withColumnRenamed("RAJ2000", "ra")
                      .withColumnRenamed("DEJ2000", "dec")
                      .withColumnRenamed("E(B-V)", "EBV") )
catalog.save_axs_table(renamed_df, "sesar_rrlyrae", calculate_zone=True)